In [1]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re

api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
issues = api.download_issues(project_id="pbssoftware")
issues_30 = [i for i in issues.values() if i.sprint_milestone == "3.0"]

In [24]:
issues_30 = [i for i in issues.values() if i.sprint_milestone == "3.0" and i.tracker == "Specification"]
print(len(issues_30))
def get_issues():
    return issues_30.copy()

103


In [15]:
for i in get_issues():
    exp = "*+FRS%d.1+*" % i.id
    d = i.description
    if d.find(exp) == -1:
        print(i.id)

3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
989
3083
3087
3090
3093
3101
3102
3106
3107
3108
3109
3114
3118
1885
3120
3122
3124
3130
3131
3132
3133
3134
3135
3136
2115
3141
3145
2231
3150
3152
3153
3154
3155
3156
3157
3158
3159
3161
3163
3164
3165
3166
3167
3168
3175
3176
3177
3178
3183
3186
3188
3189
3191
3192
3194
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3209
3210
3211
3212
3213
3214
3215
3216
3217
3218
1902
3223
3224
3225
3226
3229
3230
3231
3233
3234
3235
3236
3237
3238
3240
3242
3244
3245
3246
3247
3248
3249
3250
3252
3253
3254
3255
3258
3259
3260
3261
3264
3266
3267
3268
3269
3272
2251
3280
3287
3288
3289
3290
1262
2303
2304
2309
2316
2320
2321
1953
2367
396
2394
2395
2404
2415
2445
2458
1263
2460
2230
420
2469
2476
2484
1583
2500
2502
2516
2522
1073
2592
2596
2626
2031
2660
2669
2681
2684
2703
2711
2712
2713
2715
2716
1694
2725
2726
2733
2734
2735
2747
2752
2757
2758
2767
2769
1749
1754
2780
1759
1572
2787
2789
2794
2796
2801
2805
1783
2808
2809
2810
2812
2814
281

In [25]:
frs_re = re.compile(r"[\*\+]+(FRS[\d\.]*)[\*\+]+")
web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
for i in get_issues():
    if not frs_re.search(i.description):
        if web_frs_re.search(i.description):
            print("Goddamit Christian: %d" % i.id)
        else:
            print("Goddamnit Nathan: %d"%i.id)

Goddamnit Nathan: 1885
Goddamnit Nathan: 3122
Goddamnit Nathan: 3134
Goddamnit Nathan: 2231
Goddamnit Nathan: 3165
Goddamnit Nathan: 3176
Goddamnit Nathan: 3178
Goddamnit Nathan: 3194
Goddamnit Nathan: 3223
Goddamnit Nathan: 3225
Goddamnit Nathan: 3230
Goddamnit Nathan: 3240
Goddamnit Nathan: 3260
Goddamnit Nathan: 2309
Goddamnit Nathan: 2394
Goddamnit Nathan: 2395
Goddamnit Nathan: 2415
Goddamnit Nathan: 2669
Goddamnit Nathan: 2703
Goddamnit Nathan: 2715
Goddamnit Nathan: 1783
Goddamnit Nathan: 2808
Goddamnit Nathan: 2809
Goddamnit Nathan: 2840
Goddamnit Nathan: 2035
Goddamnit Nathan: 3008
Goddamnit Nathan: 3057
Goddamnit Nathan: 3058
Goddamit Christian: 3066
